In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import random
import time

In [2]:
# Get the urls of every constituency
result = requests.get("https://www.bbc.com/news/politics/constituencies")

In [3]:
constituency_urls = []


In [4]:
# Get the urls of every constituency
content = result.content
soup = BeautifulSoup(content, "lxml")
table = soup.findAll('table')

In [5]:
for element in table:
    a_elements = element.findAll('a')
    for a in a_elements:
        constituency_urls.append("https://www.bbc.com"+a['href'])
        
constituency_urls = list(dict.fromkeys(constituency_urls))    
print(len(constituency_urls))

constituency_urls     

650


['https://www.bbc.com/news/politics/constituencies/W07000049',
 'https://www.bbc.com/news/politics/constituencies/W07000058',
 'https://www.bbc.com/news/politics/constituencies/S14000001',
 'https://www.bbc.com/news/politics/constituencies/S14000002',
 'https://www.bbc.com/news/politics/constituencies/S14000058',
 'https://www.bbc.com/news/politics/constituencies/S14000003',
 'https://www.bbc.com/news/politics/constituencies/E14000530',
 'https://www.bbc.com/news/politics/constituencies/E14000531',
 'https://www.bbc.com/news/politics/constituencies/E14000532',
 'https://www.bbc.com/news/politics/constituencies/W07000043',
 'https://www.bbc.com/news/politics/constituencies/E14000533',
 'https://www.bbc.com/news/politics/constituencies/S14000004',
 'https://www.bbc.com/news/politics/constituencies/W07000057',
 'https://www.bbc.com/news/politics/constituencies/S14000005',
 'https://www.bbc.com/news/politics/constituencies/E14000534',
 'https://www.bbc.com/news/politics/constituencies/E140

In [39]:
import re

voting_data = []

counter = 0
for url in constituency_urls:
    constituency_result = requests.get(url)
    constituency_content = constituency_result.content

    soup = BeautifulSoup(constituency_content, "lxml")
    
    
    table = soup.findAll("table", {"class": "results-table results-table--constituency-region"})
    constituency_name = soup.find("h1", {"class":"constituency-title__title"})
    candidates_data_2019 = soup.find("ol", {"class": "ge2019-constituency-result__list"})
    candidates_data_2017 = table[0].findAll("tr", {"class": "results-table__body-row"})
    candidates_data_2015 = soup.find("div", {"class": "parties"})
    
    is_first_2019 = True
    is_first_2017 = True
    is_first_2015 = True

    print("CHECK")
    for candidate in candidates_data_2019.find_all("li"):
        try:
            print(is_first_2019)
            candidate_name = candidate.find("span", {"class": "ge2019-constituency-result__candidate-name"}) 
            candidate_name = candidate_name.text
            print(candidate_name)
            party_short = ""
            party_long = candidate.find("span", {"class": "ge2019-constituency-result__party-name"}).text
            print(party_long)
            vote_information = candidate.find("span", {"class": "ge2019-constituency-result__details-value"}).text
            print(vote_information)
            vote_percentage = candidate.findAll("span", {"class": "ge2019-constituency-result__details-value"})[1].text
            print(vote_percentage)
            voting_data.append([constituency_name.text, candidate_name, party_short, party_long, vote_percentage, 2019, is_first_2019])
            
            if is_first_2019 == False:
                break   
            is_first_2019 = False;
    
        except:
            print("")
        
    for candidate in candidates_data_2017:
        name = candidate.find("span", {"class": "results-table__body-text"})
        party_short = candidate.find("p", {"class": "results-table__party-name-const-region--short"})
        party_long = candidate.find("p", {"class": "results-table__party-name-const-region--long"})
        vote_information = candidate.findAll("td", { "class": "results-table__body-item results-table__body-item--constituency"})
        vote_percentage = vote_information[1].find("span", {"class": "results-table__body-text"})
        voting_data.append([constituency_name.text, name.text, party_short.text, party_long.text, vote_percentage.text, 2017, is_first_2017])
        if is_first_2017 == False:
            break   
        is_first_2017 = False;
                                                   
    for candidate in candidates_data_2015:
        if len(candidate) > 1:
            name = candidate.find("div", {"class": "party__name--candidate"}) 
            candidate_name = name.text
            candidate_name = candidate_name.replace("with","")
            candidate_name = candidate_name.replace("candidate","")
            
            party_short = candidate.find("div", {"class": "party__name--short"})
            party_long = candidate.find("div", {"class": "party__name--long"})
            vote_information = candidate.find("ul", { "class": "party__result"})
            vote_percentage = vote_information.find("li", {"class": "party__result--votesshare essential"})
            non_decimal = re.compile(r'[^\d.]+')
            voting_data.append([constituency_name.text, candidate_name[2:], party_short.text, party_long.text, non_decimal.sub('', vote_percentage.text), 2015, is_first_2015])                                          


            if is_first_2015 == False:
                break

            is_first_2015 = False;
               
    counter += 1
    break
    
    if (counter % 100) == 1:
        with open('voting_date'+str(counter)+'.pkl', 'wb') as f:
            pickle.dump(voting_data, f)
            print(counter)
            
    time.sleep(random.random())
        
print("DONE!")

CHECK
True
Stephen Kinnock
Labour
17,008
53.8
False
Irrelevant list item
False
Irrelevant list item
False
Irrelevant list item
False
Charlotte Lang
Conservative
6,518
20.6
DONE!


In [40]:
df = pd.DataFrame(voting_data)
df.to_pickle('election_results_uk_2015_2017_2019.pkl')
df.to_csv('election_results_uk_2015_2017_2019.csv')
df

,0,1,2,3,4,5,6
0,Aberavon,Stephen Kinnock,,Labour,53.8,2019,True
1,Aberavon,Charlotte Lang,,Conservative,20.6,2019,False
2,Aberavon,Stephen Kinnock,LAB,Labour,68.1,2017,True
3,Aberavon,Sadie Vidal,CON,Conservative,17.7,2017,False
4,Aberavon,Stephen Kinnock,(LAB),Labour,48.9,2015,True
5,Aberavon,Peter Bush,(UKIP),UKIP,15.8,2015,False


In [8]:
# len(constituency_urls)